<a href="https://colab.research.google.com/github/kchenTTP/python-series/blob/main/intermediate_python_functions/Intermediate_Python_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Intermediate Python Functions**

In our fundamentals class, we explored how to reuse code by defining functions. Now, let's dive deeper into how we can enhance our functions for more complex tasks. This is the first of two classes that will focus on intermediate to advanced function concepts.

**Table of Contents**

- [Default Parameters](#scrollTo=K3MSR-8x31WE)
- [Variable-Length Arguments](#scrollTo=Fkz86bj94ARo)
- [Type Hints & Function Documentation](#scrollTo=nQ18LxcjJH90)
  - [Type Hints](#scrollTo=87tpeu6uiwir)
  - [Docstrings](#scrollTo=vVNXZ9nwYeFc)
- [Functional Programming Concepts](#scrollTo=0GiASWE7xRbs)
  - [Lambda Functions](#scrollTo=-F-K0eG5xRLz)
  - [Higher-Order Functions](#scrollTo=XR0znysK2u90)

These topics will not only help you write cleaner, more efficient code but also make your functions more powerful and adaptable in a variety of programming scenarios. Let's get started!



## **Default Parameters**

Default parameters allow you to define functions where some arguments have predetermined values if not specified by the caller. This makes functions more flexible and easier to use while maintaining backward compatibility.

**Key Concepts**

- Default parameters must come after non-default parameters
- Default values are evaluated when the function is defined, not when it's called
- Mutable default arguments can cause unexpected behavior


In [ ]:
# Default parameter after required parameter
def greet(name, greeting="Hello"):
  return f"{greeting}, {name}!"

In [ ]:
# BAD: Default parameter before required parameter
def bad_greet(greeting="Hello", name):  # This will cause a SyntaxError
  return f"{greeting}, {name}!"

SyntaxError: non-default argument follows default argument (<ipython-input-4-0fefceb40d24>, line 2)

💡 When using a function with default parameters, the parameters that have default values are optional. If you choose to pass an argument for these parameters, the provided value will override the default one.

In [ ]:
# Call function with default value
print(greet("Alice"))

# Override default value
print(greet("Bob", "Hi"))
print(greet("Tim", "G'day"))

Hello, Alice!
Hi, Bob!
G'day, Tim!


### **Common Pitfall: Mutable Default Arguments**

When using mutable default arguments, such as lists or dictionaries. If you modify the mutable object within the function, that change will persist across subsequent calls to the function. This can lead to unexpected behavior, as the default argument will retain its modified state instead of reverting to the initial value.

Take a look at the below example:

In [ ]:
def duplicated_values(lst, duplicated=[], seen=[]):
  for item in lst:
    if item in seen:
      duplicated.append(item)
      continue
    seen.append(item)
  return {
      "duplicated": duplicated,
      "seen": seen
  }

# First call
print(duplicated_values([1, 2, 3, 2]))
# Second call
print(duplicated_values([4, 4, 5, 6]))

{'duplicated': [2], 'seen': [1, 2, 3]}
{'duplicated': [2, 4], 'seen': [1, 2, 3, 4, 5, 6]}


As you can see, in our second function call, even though we intended to create a new list with different values, the values from our first function call still appeared.

This is because the mutable default argument (the list) retains any changes made to it across all calls to the function. Instead of starting fresh with each call, it continues using the modified list from previous calls.

<br>

To avoid this pitfall, it's recommended to use `None` as a default value and then create a new mutable object inside the function if needed.

In [ ]:
def duplicated_values_fixed(lst, duplicated = None, seen = None):
  if duplicated is None:
    duplicated = []
  if seen is None:
    seen = []

  for item in lst:
    if item in seen:
      duplicated.append(item)
      continue
    seen.append(item)
  return {
      "duplicated": duplicated,
      "seen": seen
  }

# First call
print(duplicated_values_fixed([1, 2, 3, 2]))
# Second call
print(duplicated_values_fixed([4, 4, 5, 6]))

{'duplicated': [2], 'seen': [1, 2, 3]}
{'duplicated': [4], 'seen': [4, 5, 6]}


This approach ensures that each call to `duplicated_values_fixed()` creates a new list if one isn't provided, keeping the function's behavior consistent and free from unintended side effects.

### **Exercise**

Create a function that formats a price with optional currency and decimal places.

> Currency symbol example: $, €, ¥, £


In [ ]:
# Your code Goes here


In [ ]:
# @title Solution

def format_price(price, currency="$", decimal_places=2):
  return f"{currency}{price:.{decimal_places}f}"


print(format_price(24.99))
print(format_price(24.99, "€"))
print(format_price(24.99, "¥", 0))

## **Variable-Length Arguments**

Python provides two special parameters for handling variable numbers of arguments:

- `*args`: Collects additional positional arguments into a ***tuple***
- `**kwargs`: Collects additional keyword arguments into a ***dictionary***

**Key Concepts**

- `*args` allows passing any number of positional arguments
- `**kwargs` allows passing any number of keyword arguments
- Convention is to use `args` and `kwargs`, but any valid variable name works
- The order or argument matters:

  > `regular args` → `*args` → `default args` → `**kwargs`


> 🚨 Typically, when you pass arguments with their parameter names, like `func(arg1=value)`, the order doesn't matter because each argument is explicitly tied to its parameter. However, if you mix positional arguments (`*args`) and keyword arguments (`**kwargs`), order becomes important.
>
> Positional arguments must come before keyword arguments in the function call, or you'll encounter an error.


In [ ]:
# Argument ordering
def example_function(required, *args, default="default", **kwargs):
  print(f"Required: {required}")
  print(f"Args: {args}")
  print(f"Default: {default}")
  print(f"Kwargs: {kwargs}")

# Usage
example_function("required", 1, 2, 3, default="custom", extra1="value1", extra2="value2")

Required: required
Args: (1, 2, 3)
Default: custom
Kwargs: {'extra1': 'value1', 'extra2': 'value2'}


You can access `*args` using indices, as it is stored as a tuple, and you can access `**kwargs` using dictionary methods like `.get()` or `.items()` since it's stored as a dictionary.

In [ ]:
# Accessing args and kwargs
def example_function_2(*args, **kwargs):
    # Accessing args
    print(args[0])  # Access the first positional argument

    # Accessing kwargs
    print(kwargs.get('name'))  # Access the 'name' keyword argument if it exists
    print(kwargs.items())  # Get all key-value pairs as a list of tuples


example_function_2(1, 2, 3, name="Alice", age=25)

1
Alice
dict_items([('name', 'Alice'), ('age', 25)])


For example we could write a `sum()` function that takes in a variable amount of numbers and sum them all up:

In [ ]:
def sum(*args):
  total = 0
  for num in args:
    total += num
  return total

print(sum(1, 2, 3))
print(sum(4, 5, 6, 7))

6
22


### **Keyword-Only Arguments**

You can use `*` to enforce that any parameter defined after `*` must be passed as a keyword argument.

This means you must specify the parameter name when passing arguments to these parameters; otherwise, you'll get an error.

> 📒 **Note**: This approach is especially useful for improving code readability and preventing accidental errors in function calls.


In [ ]:
def greet_modified(name, *, greeting="Hello"):
  return f"{greeting}, {name}!"

print(greet_modified("Alice"))
print(greet_modified("Bob", greeting="Hi"))

Hello, Alice!
Hi, Bob!


In [ ]:
# Not allowed: Didn't provide parameter name to keyword-only argument
print(greet_modified("Tom", "Morning"))

TypeError: greet_modified() takes 1 positional argument but 2 were given

In [ ]:
# Not allowed: Arguments in incorrect order
print(greet_modified(name="Tom", "Morning"))

SyntaxError: positional argument follows keyword argument (<ipython-input-37-19dfc297c491>, line 2)

### **Exercise: Generate API Endpoints**

Many APIs allow you to pass multiple parameters to customize requests.

Let's write a function that takes:
1. a base API endpoint as argument
2. your API key as argument
3. a variable number of API parameters as keyword arguments

This function will then generate a complete API URL with the parameters included.

> Hint: Use `?` after the base API endpoint before passing in the parameters and use `&` for multiple API parameters
>
> Example: `https://www.google.com/search?q=apple&sourceid=chrome&ie=UTF-8`



In [ ]:
example_endpoint = "https://example.com/api"

In [ ]:
# Your code goes here


In [ ]:
# @title Solution

def generate_api_endpoint(base_url, api_key, **params):
  url = f"{base_url}?api_key={api_key}"
  for key, value in params.items():
    url += f"&{key}={value}"
  return url


api_key = "my_api_key_123"
url = generate_api_endpoint(example_endpoint, api_key, format="json", limit=10)
print(url)

## **Type Hints & Function Documentation**


### **Type Hints**

Type hints provide a way to annotate code with expected types.

<br>

**Benefits include:**

- **Self-documenting code**
  
  Clarifies what types are expected, making code easier to read.

- **IDE support and code completion**

  Enhances code suggestions and error detection in many code editors.

- **Static type checking**

  Tools like [mypy](https://mypy.readthedocs.io/en/stable/index.html) can catch type-related errors before runtime.

- **Ease of understanding and refactoring**

  Simplifies the process of understanding code, especially in larger projects.

<br>

**Functions without type hints have:**

- **Reduced Readability**

  Functions without type hints can be harder to read and understand, as the expected types for parameters and return values are not explicitly stated.

- **Limited IDE Support**

  Functions lacking type hints cannot fully leverage IDE tooltip functionality, which provides insights into function usage and parameter data types.

- **No Static Error Checking**

  Linters cannot check for potential type-related errors, making it easier to overlook mistakes that could lead to runtime issues.

<br>

> 📒 **Note**: Type hints do not change the functionality of your code and are completely optional (except in some libraries, such as [FastAPI](https://fastapi.tiangolo.com/), where they enable features such as data validation and automatic documentation). However, including them can significantly improve the readability and maintainability of your code.

<br>

For more in-depth information on writing type hints, check out [Python's documentation](https://peps.python.org/pep-0484/)


Let's look at how we can write type hints ourself.


In [ ]:
# Basic type hint example
def process_string(text: str, times: int = 1) -> str:
  return text * times

def cube(number: float) -> float:
  return number ** 3

number = input("Enter a number: ") # input() function returns a string

# We get a red squiggly line from the code editor because this will raise an error: strings cannot be used in arithmetic operations.
print(cube(number))

Enter a number: 3


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

In [ ]:
# No type hint example
def cube_(number):
  return number ** 3

number = input("Enter a number: ") # input() function returns a string

# We don't get a red squiggly line from the code editor because it doesn't know what data type to expect even though this will clearly error
print(cube_(number))

Enter a number: 2


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

If our function or variable can take or return multiple data types, we can use the `|` operator to separate multiple types:


In [ ]:
# Multiple acceptable types
def is_even(num: int | float) -> bool:
  return num % 2 == 0

We can even write type hints for variables

> 🚨 There is no need to do this for every variable but for obscure ones where the datatype of the variable is very hard to infer, this is extremely useful!


In [ ]:
# Type hinting variables
num_1: int = 1
num_2: float = 1.0

var_3 = {"a": 1, "b": 2, "c": 3, "d": 4}
num_3: int | None = var_3.get("apple") # in this example get() can either return an 'int' or 'None'. Using type hint makes it clearer to see what kind of datatype you are getting

#### Type Hints for Complex Data Types

For more complex data types, you may need to import specific classes from the Python standard library or third-party libraries, or you can define your own custom data types.

Here are a couple examples:

In [ ]:
class CustomType:
  def do_sth(self) -> None:
    pass

def construct_custom_type() -> CustomType:
  return CustomType()

In [ ]:
from collections.abc import Iterable

def print_iterable(iterable: Iterable) -> None:
  if not isinstance(iterable, Iterable):
    raise TypeError("Expected an iterable")

  for item in iterable:
    print(item)


print_iterable([1, 2, 3])
print_iterable((4, 5, 6))

1
2
3
4
5
6


In [ ]:
from typing import Literal

def choose_color(color: Literal["red", "green", "blue"]) -> None:
  if not isinstance(color, str): raise TypeError("Expected a string")
  if not color.lower() in ["red", "green", "blue"]: raise ValueError("Invalid color")

  match color:
    case "red":
      pass
    case "green":
      pass
    case "blue":
      pass
    case _:
      print("Shouldn't be able to reach here, something is very wrong!")
  print(f"You chose {color}")

choose_color("red")
choose_color("green")
choose_color("blue")

You chose red
You chose green
You chose blue


In [ ]:
from collections.abc import Callable
from typing import Any

def repeat_func(func: Callable, times: int, *args: Any, **kwargs: Any) -> None:
  for _ in range(times):
    rv = func(*args, **kwargs)
    print(rv)


repeat_func(greet, times=2, name="Timmy", greeting="What's up")
repeat_func(cube, times=3, number=2)

What's up, Timmy!
What's up, Timmy!
8
8
8


In [ ]:
from typing import Optional, Union

def process_data(items: list[dict[str, Union[str, int]]], config: Optional[dict[str, str]] = None) -> tuple[list[str], int]:
  result = []
  config = config or {}  # Use empty dict if config is None

  for item in items:
    # Get the field to process from config, default to 'name'
    field = config.get('field', 'name')

    # Get the value, convert to uppercase if it's a string
    value = item.get(field, '')
    if isinstance(value, str):
      value = value.upper()

    result.append(str(value))

  return result, len(result)


# Example:
items = [
    {'name': 'alice', 'age': 30},
    {'name': 'bob', 'age': 25},
    {'name': 'charlie', 'age': 35}
]

# Process names
names, count = process_data(items)
print(names)
print(count)

# Process ages with config
ages, count = process_data(items, {'field': 'age'})
print(ages)
print(count)

['ALICE', 'BOB', 'CHARLIE']
3
['30', '25', '35']
3


In [ ]:
import pandas as pd

def create_df(data: dict) -> pd.DataFrame:
  return pd.DataFrame(data)


data = {
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [25, 30, 35],
    'city': ['New York', 'London', 'Paris']
}

df = create_df(data)
df

,name,age,city
0,Alice,25,New York
1,Bob,30,London
2,Charlie,35,Paris


### **Docstrings**

Docstrings provide a standardized way to document Python functions and can serve multiple purposes, including:

- Clear documentation for other developers
- Interactive help, such as through the help() function in Python or Language Server Protocols (LSPs) in code editors
- Source for automatic documentation generators like [Sphinx](https://www.sphinx-doc.org/en/master/index.html) and [Swagger](https://swagger.io/docs/).

A well-structured docstrings can improve code readability and make maintenance a lot easier. Here are a few popular docstring styles to consider:

<br>

**Common Docstring Styles**

- [Google Style](https://github.com/google/styleguide/blob/gh-pages/pyguide.md#38-comments-and-docstrings): Known for its readability, with parameters and return values clearly outlined.
- [NumPy Style](https://numpydoc.readthedocs.io/en/latest/format.html#docstring-standard): Particularly useful in scientific and data-centric projects, with structured sections for extended explanations.
- [reStructuredText Style](https://sphinx-rtd-tutorial.readthedocs.io/en/latest/docstrings.html): Often used with Sphinx, this style supports extensive formatting options and compatibility with reStructuredText-based documentation tools.

<br>

For more in-depth information on writing effective docstrings, check out this [article on docstrings](https://www.dataquest.io/blog/documenting-in-python-with-docstrings/).


In [ ]:
# Google Style Example
def scream(some_string: str) -> None:
  """Scream out the input string.

  Converts all characters in a string to uppercase to simulate screaming. The longer the string, the more exclamation marks will be added at the end.

  Args:
    some_string (str): The string to convert to uppercase.

  Returns:
    None

  Raises:
    TypeError: If the input is not a string.
    ValueError: If the input is an empty string.

  Examples:
    >>> scream_at_my_face("go home denis, you are drunk")
    GO HOME DENIS, YOU ARE DRUNK!!!!!!!!!!!!!!
  """
  if not isinstance(some_string, str):
    raise TypeError(f"Expected a string, instead got: {type(some_string)}")

  some_string = some_string.strip()

  if not some_string:
    raise ValueError("Expected a non-empty string")

  volume = max(len(some_string) // 2, 1)
  print(f"{some_string.upper()}{'!' * volume}")

Now, if you use the `help()` function or hover your mouse over the function name in the code cell or an editor, the tooltip will show the function signature along with the docstring.

This makes it much easier to quickly understand what the function does, its parameters, and any exceptions it may raise.


In [ ]:
help(scream)

Help on function scream in module __main__:

scream(some_string: str) -> None
    Scream out the input string.
    
    Converts all characters in a string to uppercase to simulate screaming. The longer the string, the more exclamation marks will be added at the end.
    
    Args:
      some_string (str): The string to convert to uppercase.
    
    Returns:
      None
    
    Raises:
      TypeError: If the input is not a string.
      ValueError: If the input is an empty string.
    
    Examples:
      >>> scream_at_my_face("go home denis, you are drunk")
      GO HOME DENIS, YOU ARE DRUNK!!!!!!!!!!!!!!



In [ ]:
# Try hovering your mouse over the the function name
scream("hello")
scream("Mr. Potato Head, this is the last time I am going to tell you this. Stop removing your eyes")

HELLO!!
MR. POTATO HEAD, THIS IS THE LAST TIME I AM GOING TO TELL YOU THIS. STOP REMOVING YOUR EYES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


## **Functional Programming Concepts**



### **Lambda Functions**

Lambda functions are small, anonymous functions that can have any number of arguments but can only have one expression. They are useful for short operations that don't need a full function definition.

**Key Concepts**

- **Single Expression Only**

  A lambda function can only contain one expression, which is evaluated and returned.

- **Can be used as function arguments**

  Lambda functions can be passed as arguments to higher-order functions like `map()`, `filter()`, and `sorted()`.
  
- **Useful for sorting and filtering**

  They are often used for single usage operations like sorting lists or filtering data without needing a named function.

- **Should be simple and readable**

  Lambda functions are best for simple operations; complex logic should be implemented in a regular function for clarity.

<br>

**Syntax**

```python
lambda arguments: expression
```


To show how lambda functions truely work underneath the hood. Here's a few lambda functions and their equivalent functions if they were defined like a regular function:

In [ ]:
def square_verbose(x):
  return x ** 2

square = lambda x: x ** 2

print(square_verbose(3))
print(square(3))

9
9


In [ ]:
def get_max_verbose(a, b):
  return a if a > b else b

get_max = lambda a, b: a if a > b else b

print(get_max_verbose(1, 2))
print(get_max(1, 2))

2
2


We can rewrite our `greet()` function to a lambda function that takes multiple arguments since it's a single expression function:


In [ ]:
# Lambda with multiple arguments
greet_l = lambda name, greeting: f"{greeting}, {name}!"

print(greet_l("Alice", "Hello"))
print(greet_l("Bob", "Hi"))

Hello, Alice!
Hi, Bob!


Here are a couple more examples for lambda functions:


In [ ]:
# Lambda with conditional logic
is_adult = lambda age: "Adult" if age >= 18 else "Minor"
print(is_adult(25))
print(is_adult(15))

Adult
Minor


In [ ]:
# Lambda with list comprehension
format_names = lambda lst: [str(x).title() for x in lst]
print(format_names(["alice", "BOB", "ChArLiE"]))

flatten_list = lambda lst: [item for sublist in lst for item in sublist]
print(flatten_list([[1, 2, 3], [4, 5], [6, 7, 8, 9]]))

['Alice', 'Bob', 'Charlie']
[1, 2, 3, 4, 5, 6, 7, 8, 9]


> 💡 Lambda functions are most commonly used with [higher-order functions](#scrollTo=XR0znysK2u90), which we will cover in the next section.


#### **Example: Sort Dictionaries by Keys**

Let's look at an example where we create a function that sorts a list of dictionaries of simliar construct (same keys) by the specified key.


In [ ]:
def sort_objects(objects: list[dict[str, Any]], sort_key: str, reverse: bool=False) -> list[dict]:
  """Sort a list of dictionaries by a single key.

  Args:
    objects (list[dict[str, Any]]): List of dictionaries to sort
    sort_key (str): The dictionary key to sort by
    reverse (bool): Whether to reverse the sort

  Returns:
    Sorted list of dictionaries
  """
  return sorted(
      objects,
      key=lambda x: x.get(sort_key, ""), # This will return the value of the sort_key which the sorted function will sort base on the values
      reverse=reverse
  )


data = [
    {"name": "Timmy", "age": 35, "salary": 60000},
    {"name": "John", "age": 25, "salary": 55000},
    {"name": "Andrea", "age": 30, "salary": 75000}
]

print("Sort by age:", sort_objects(data, "age"))
print("Sort by name:", sort_objects(data, "name"))
print("Sort by salary (rev):", sort_objects(data, "salary", reverse=True)) # Sort by salary (descending)

Sort by age: [{'name': 'John', 'age': 25, 'salary': 55000}, {'name': 'Andrea', 'age': 30, 'salary': 75000}, {'name': 'Timmy', 'age': 35, 'salary': 60000}]
Sort by name: [{'name': 'Andrea', 'age': 30, 'salary': 75000}, {'name': 'John', 'age': 25, 'salary': 55000}, {'name': 'Timmy', 'age': 35, 'salary': 60000}]
Sort by salary (rev): [{'name': 'Andrea', 'age': 30, 'salary': 75000}, {'name': 'Timmy', 'age': 35, 'salary': 60000}, {'name': 'John', 'age': 25, 'salary': 55000}]


### **Higher-Order Functions**

Higher-order functions are functions that can either:

- Accept other functions as arguments
- Return functions as results
- Perform both of the above

Here are a few examples of higher-order functions:


#### **Example 1: BYOP (Bring Your Own *(string)* Processing)**

This function takes a string and applies a specified function to it, along with any additional arguments.

<br>

**Components**

-	**Parameters**
	-	*`text` (str):* The input string to be processed.
	-	*`func` (Callable):* A function that will be applied to `text`. The function should take a string as its first argument and return a modified string.
	-	_`*args`:_ Additional positional arguments that may be required by `func`.
	-	*`**kwargs`:* Additional keyword arguments that may be required by `func`.

-	**Returns**
	-	A processed string, which is the result of applying `func` to `text` with any optional arguments.

<br>

**Explanation**

1.	Applying the Function:

  `process_string` calls the function `func` on `text`, passing any extra `*args` or `**kwargs` as needed by the function. This makes `process_string` adaptable to different kinds of string processing functions.


In [ ]:
# Process a string with your function of choice
def process_string(text: str, func: Callable[[str], str], *args, **kwargs) -> str:
  return func(text, *args, **kwargs)

print(process_string("hello world", str.upper))
print(process_string("tiMoTHy", str.title))
print(process_string("hello", lambda x: x[::-1])) # Reverse a string
print(process_string("a quick brown fox jumps over another fox", str.replace, "fox", "dog"))

HELLO WORLD
Timothy
olleh
a quick brown dog jumps over another dog


#### **Example 2: Create Logger**

This function creates custom logging functions with flexible log levels and timezone options to generate time-stamped log messages with a consistent format.

<br>

**Components**

-	**Parameters**
	- *`log_level` (str):* Specifies the log level or prefix (e.g., "INFO", "WARNING").
  - *`timezone` (str):* An optional parameter with a default value of "America/New_York" that sets the timezone for the timestamp using `pytz`.
	- *`cache_len` (list):* A mutable list that stores the length of the longest `log_level`. This ensures all log entries align properly by standardizing the width of the log level display. A list of one integer can be provided to modify the prefix width but it needs to be larger than the length of `log_level`.

- **Returns**
	- A logger function, which takes a single `message` parameter (the log message) and prints it with a formatted timestamp and log level prefix.

<br>

**Explanation**

1.	Updating `cache_len` for Alignment:

  `cache_len` is initialized as an empty list, storing the longest `log_level` length found so far. Using a mutable default argument allows each call to `create_logger` to share this list and alignment using the proper character length.

	-	If `cache_len` is empty, it initializes with the current `log_level` length.
	-	If `cache_len` already has a value, it updates only if the new `log_level` length is greater than the stored length.

2.	Defining the `logger` Inner Function:

	This inner function `logger` formats and prints log messages using:

    -	`log_level`, left-aligned according to `cache_len[0]` to ensure consistent width across different loggers.
    -	`current_time`, a timestamp based on the `timezone` argument.
    -	`message`, the message to be logged.

3.	Returning the `logger` Function:

  The `create_logger` function returns this `logger` function, which can be used as a customized logger with a prefix.


In [ ]:
from datetime import datetime
import pytz

def create_logger(log_level: str, timezone: str="America/New_York", cache_len: list=[]) -> Callable[[str], None]:
  length = len(log_level)
  if not cache_len:
    cache_len.append(length)
  else:
    if length > cache_len[0]:
      cache_len[0] = length

  def logger(message: str) -> None:
    current_time = datetime.strftime(datetime.now(tz=pytz.timezone(timezone)), "%Y-%m-%d %H:%M:%S")
    print(f"{log_level:<{cache_len[0]}} - {current_time} - {message}")

  return logger


info_logger = create_logger("INFO")
warning_logger = create_logger("WARNING")

info_logger("This is an info message")
warning_logger("This is a warning message")

INFO    - 2024-10-31 20:42:06 - This is an info message
WARNING - 2024-10-31 20:42:06 - This is a warning message


#### **Common Higher-Order Functions in Python**

- `map()`
- `filter()`
- `reduce()`

> 💡 These functions are very often used with [lambda functions](#scrollTo=-F-K0eG5xRLz)


##### **`map()`**

Applies a function to every item in an iterable, creating a new iterable with the results. Useful for transforming data without using loops.


In [ ]:
# Squaring a list of numbers
numbers = [1, 2, 3, 4, 5]
squared_numbers = list(map(lambda x: x ** 2, numbers))
print(squared_numbers)

[1, 4, 9, 16, 25]


In [ ]:
# Process a list of names
names = ["Janice", "Allen  ", "\nBarba"]
processed_names = list(map(lambda x: x.strip().upper(), names))
print(processed_names)

['JANICE', 'ALLEN', 'BARBA']


##### **`filter()`**

Selects items from an iterable based on a function that returns `True` or `False`, returning only items that meet the condition.


In [ ]:
# Filter even numbers
numbers = [1, 2, 3, 4, 5]
even_numbers = list(filter(lambda x: x % 2 == 0, numbers))
print(even_numbers)

# Filter number greater than or equal to 3
filtered_numbers = list(filter(lambda x: x >= 3, numbers))
print(filtered_numbers)

[2, 4]
[3, 4, 5]


In [ ]:
# Get people who attended an event
def attended_event(attendees: list[dict[str, Any]]) -> list[dict[str, str]]:
  attended = list(filter(lambda x: x["attended"], attendees))
  return [p["name"] for p in attended]

# Get people who are late
def is_late(attendees: list[dict[str, Any]], scheduled_time: str) -> list[dict[str, str]]:
  hr, min = scheduled_time.split(":")
  attended = list(filter(lambda x: x["attended"], attendees))
  arrived_same_hr = list(filter(lambda x: int(x["time_arrived"].split(":")[0]) >= int(hr), attended))
  late = list(filter(lambda x: int(x["time_arrived"].split(":")[1]) > int(min), arrived_same_hr))
  return [p["name"] for p in late]

attendees = [
    {"name": "Andy", "attended": True, "time_arrived": "11:34"},
    {"name": "Joclyn", "attended": True, "time_arrived": "12:00"},
    {"name": "Chandler", "attended": True, "time_arrived": "12:01"},
    {"name": "David", "attended": False, "time_arrived": ""}
]

present_attendees = attended_event(attendees)
print(present_attendees)

late_attendees = is_late(attendees, scheduled_time="12:00")
print(late_attendees)

['Andy', 'Joclyn', 'Chandler']
['Chandler']


##### **`reduce()` (Additional Material)**

The `reduce()` function successively reduces an iterable to a single cumulative value by applying a specified function to pairs of items. The function used with reduce must accept two positional arguments, as it operates on pairs of items in sequence.

It is commonly used for accumulating data, such as summing, multiplying, or otherwise combining items.

> 📒 **Note:** reduce() requires importing from the functools module.

**Syntax**

```python
reduce(lambda arg1, arg2: expression, Iterable)
```


In [ ]:
from functools import reduce

In [ ]:
# Multiply all the numbers in the list
numbers = [2, 3, 2, 5]
product = reduce(lambda x, y: x * y, numbers)
print(product)

60


Here's how the cell above works:

1. `reduce` applies a function cumulatively to items in an iterable, from left to right, to reduce them to a single value.
2. Lambda Function: The `lambda x, y: x * y` function takes two parameters, `x` and `y`, and multiplies them. This function will be applied to each pair of numbers in numbers in sequence. `x` is the result of the lambda function for the previous pair.
3. Reducing the List:
  - `reduce` begins with the first two items in numbers (2 and 3) and multiplies them to get 6.
  - Then it takes 6 and the next item (2) and multiplies them, resulting in 12.
  - Finally, it takes 12 and the last item (5), multiplying them to get 60.


###### Examples
Here's a couple more examples:

> 📒 **Note:** It's okay to not understand the following examples.
> - The mental model for `reduce()` and `lambda` functions is pretty tricky.
> - Passing functions as arguments also feels weird since we're used to passing values as arguments.


In [ ]:
# Find maximum value
numbers = [3, 5, 2, 8, 1]
max_value = reduce(lambda x, y: x if x > y else y, numbers)
print(max_value)

8


In [ ]:
# Merge multiple dictionaries with conflict resolution
dicts = [
    {'a': 1, 'b': 2},
    {'b': 3, 'c': 4},
    {'c': 5, 'd': 6}
]

def merge_resolve(acc: dict, curr: dict) -> dict:
  # Unpack previous dictionary to variable d
  d = {**acc}

  # If conflicted keys, choose the larger value
  for k, v in curr.items():
    if k in d:
      d[k] = max(d[k], v)
    else:
      d[k] = v

  return d

merged = reduce(merge_resolve, dicts)
print(merged)

{'a': 1, 'b': 3, 'c': 5, 'd': 6}


In [ ]:
# Compose multiple functions into a pipeline that takes a single argument
def compose_functions(*funcs) -> Callable:
  """Compose (nest) multiple functions into a single function."""
  return reduce(lambda f, g: lambda x: f(g(x)), funcs)

to_upper = lambda s: s.upper()
remove_spaces = lambda s: s.replace(' ', '')
add_exclamation = lambda s: s + '!'

# Here we use compose_function to basically create a string processing pipeline
pipeline = compose_functions(add_exclamation, to_upper, remove_spaces)
print(pipeline("a quick brown fox jumps over a box"))

# Here's a url processing pipeline
process_url = compose_functions(
    lambda s: s.strip('/'),
    lambda s: s.replace('http', 'https'),
    lambda s: s.lower()
)
print(process_url("HTTP://WWW.GOOGLE.COM/"))

AQUICKBROWNFOXJUMPSOVERABOX!
https://www.google.com


In [ ]:
# @title Verbose version of the above `compose_functions` example
def verbose_pipeline(*funcs):
  def compose(f, g):
    def composed_function(x):
      g_result = g(x)
      print(f"After {g.__name__}: {g_result}")
      f_result = f(g_result)
      print(f"After {f.__name__}: {f_result}")
      return f_result
    return composed_function

  return reduce(compose, funcs)

# Make our functions named for better debugging
def remove_spaces(s): return s.replace(' ', '')
def to_upper(s): return s.upper()
def add_exclamation(s): return s + '!'

pipeline = verbose_pipeline(add_exclamation, to_upper, remove_spaces)
print(pipeline("a quick brown fox jumps over a box"))

print("=====")
process_url = verbose_pipeline(
    lambda s: s.strip('/'),
    lambda s: s.replace('http', 'https'),
    lambda s: s.lower()
)
print(process_url("HTTP://WWW.GOOGLE.COM/"))